In [1]:
import numpy as np
from numpy.random import default_rng
import tensorflow as tf
from tensorflow import keras
import os
import PIL
import pandas as pd


In [90]:
filepath='.'
filedir=os.path.join(os.getcwd(),filepath,'Vegetable Images')
columns=['img_name','label']


In [86]:
valid_df.head(5)

,img_name,label
3000,C:\Users\Sean\Documents\GitHub\INM702-Coursewo...,Tomato
